# Vergleichen und Kontrollieren die ähnlichen Wörter von unterschiedlichen Embeddings-Methoden

1. Word2Vec-Methoden: CBOW und Skipgram möglich. Nach dem Paper benutzen wir Skipgram für unser Experiment
2. Für das neue Experiment wird BERT-Modell verwendet, um Wortembeddings zu erstellen. Das ist ein mehrschritten-Prozess: Durchschnitt. Die genaue Berechnung wird in dem Bericht beschrieben
3. Durch mehrere Schritten Durchschnitt sind die semantischen Ähnlichkeiten noch bebeihalten. 
4. Die genaue Implementierung für Embeddings in dem `src\embedding.py` und `src/bert*` zu sehen

# Datenvorbereitung

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import time

from src.prepare_dataset import TextDataLoader
# init TextDataLoader für die Datenquelle 20 News Groups
# Daten abrufen vom Sklearn, tokenisieren und besondere Charaktern entfernen
textsloader = TextDataLoader(source="20newsgroups", train_size=None, test_size=None)
textsloader.load_tokenize_texts("20newsgroups")
# Vorverarbeitung von Daten mit folgenden Schritten:
textsloader.preprocess_texts(length_one_remove=True, punctuation_lower = True, stopwords_filter = True)
# Daten zerlegen für Train, Test und Validation. Erstellen Vocabular aus dem Trainset
min_df= 30
textsloader.split_and_create_voca_from_trainset(max_df=0.7, min_df=min_df, stopwords_remove_from_voca=True)

# Erstellen BOW-Repräsentation für ETM Modell
for_lda_model = False
word2id, id2word, train_set, test_set, val_set = textsloader.create_bow_and_savebow_for_each_set(for_lda_model=for_lda_model)

loading texts: ...
train-size after loading: 11314
test-size after loading: 7532
finished load!
start: preprocessing: ...
finised: preprocessing!
vocab-size in df: 8473
start creating vocabulary ...
length of the vocabulary: 8473
length word2id list: 8473
length id2word list: 8473
finished: creating vocabulary
train-size-after-all: 11214
test-size-after-all: 7532
validation-size-after-all: 100
test-size-after-all: 11214
test-indices-length: 11214
test-size-after-all: 100
test-indices-length: 100
test-size-after-all: 7532
test-indices-length: 7532
length train-documents-indices : 1146648
length of the vocabulary: 8473


start: creating bow representation...
finised creating bow input!

start: creating bow representation...
finised creating bow input!

start: creating bow representation...
finised creating bow input!

start: creating bow representation...
finised creating bow input!

start: creating bow representation...
finised creating bow input!



In [2]:
# Kontrollieren die Größen von verschiedenen Datensätzen
print(f'Size of the vocabulary after prprocessing ist: {len(textsloader.vocabulary)}')
print(f'Size of train set: {len(train_set["tokens"])}')
print(f'Size of val set: {len(val_set["tokens"])}')
print(f'Size of test set: {len(test_set["test"]["tokens"])}')

# re-erstellen von Dokumenten nach der Vorverarbeitungen. Die Dokumenten sind in Wörtern und werden für Word-Embedding Training benutzt
docs_tr, docs_t, docs_v = textsloader.get_docs_in_words_for_each_set()
del docs_t
del docs_v
train_docs_df = pd.DataFrame()
train_docs_df['text-after-preprocessing'] = [' '.join(doc) for doc in docs_tr[:100]]
train_docs_df

Size of the vocabulary after prprocessing ist: 8473
Size of train set: 11214
Size of val set: 100
Size of test set: 7532


,text-after-preprocessing
0,biochem nwu jackson swimming pool defense nntp...
1,apollo hp red herring police state usa nntp po...
2,hades coos dartmouth brian hughes installing r...
3,jaeger buphy bu gregg jaeger rushdie boston un...
4,king eng umd doug boom computer design lab mar...
...,...
95,physics ca campbell pc windows os unix reply p...
96,ncr jim sharp parts information distribution w...
97,sera zuma serdar argic nazi germany armenians ...
98,chips astro temple charlie mathew bible resear...


# WordEmbedding mit Skipgramm

1. Trainieren Word-Embedding 
2. Speichern in txt File
3. Wiederaufrufen

In [3]:
from src.embedding import WordEmbeddingCreator

In [4]:
vocab = list(word2id.keys()) # vocabulary after preprocessing and creating bow
word2vec_model = 'skipgram'

save_path = Path.joinpath(Path.cwd(), f'prepared_data/min_df_{min_df}')
figures_path = Path.joinpath(Path.cwd(), f'figures/min_df_{min_df}')
Path(save_path).mkdir(parents=True, exist_ok=True)
Path(figures_path).mkdir(parents=True, exist_ok=True)

if word2vec_model!="bert":
    # only for cbow and skipgram model
    wb_creator = WordEmbeddingCreator(model_name=word2vec_model, documents = docs_tr, save_path= save_path)
    wb_creator.train(min_count=0, embedding_size= 300)
    wb_creator.create_and_save_vocab_embedding(vocab, save_path)

train word-embedding with skipgram


  2%|▏         | 171/8473 [00:00<00:04, 1706.03it/s]

length of vocabulary from word-embedding with skipgram: 8473
length of vocabulary after creating BOW: 8473


100%|██████████| 8473/8473 [00:04<00:00, 1847.50it/s]


In [5]:
print(f'word: {vocab[1]} - vector: {list(wb_creator.model.wv.__getitem__(vocab[0]))[:5]} ')

word: analogous - vector: [0.075638294, 0.0022103025, 0.10262413, -0.06819996, -0.077393636] 


# Vergleichen ähnliche Wörter von Word2Vec (gensim und eigene Cosine)

In [6]:
v = vocab[0]
vec = list(wb_creator.model.wv.__getitem__(v))
print(f'word-embedding of the word-- {v}: ')
print(f'dim of vector: {len(vec)}')

word-embedding of the word-- permanently: 
dim of vector: 300


In [7]:
# using gensim function
wb_creator.find_most_similar_words(n_neighbor=10, word=v)

[('varies', 0.9419652223587036),
 ('yield', 0.9410173296928406),
 ('continuous', 0.9399767518043518),
 ('minimize', 0.935795247554779),
 ('margin', 0.9349756240844727),
 ('threshold', 0.9342159032821655),
 ('additionally', 0.9321402311325073),
 ('efficiency', 0.9306280612945557),
 ('characteristic', 0.9303073883056641),
 ('functional', 0.9291945695877075)]

In [8]:
# using self-implemented cosine
wb_creator.find_similar_words_self_implemented(10, vocab, v)

{'varies': 0.9419652,
 'yield': 0.9410174,
 'continuous': 0.9399768,
 'minimize': 0.9357952,
 'margin': 0.93497556,
 'threshold': 0.9342159,
 'additionally': 0.9321402,
 'efficiency': 0.93062806,
 'characteristic': 0.93030745,
 'functional': 0.92919457}

# Vergleich ähnliche Wörter zwischen Word2Vec und Bert

In [9]:
# only bert_vocab, not embeddings in this file
with open('prepared_data/bert_vocab.txt') as f:
    lines = f.readlines()
readed_bert_vocab = [e.split("\n")[0] for e in lines]
print(len(readed_bert_vocab))

104008


In [10]:
from src.embedding import BertEmbedding
bert_eb = BertEmbedding('prepared_data') #directory, where the txt.file of bert_vocab_embedding.txt ist
bert_eb.get_bert_embeddings(vocab)

read word-embeddings with bert from file...
bert-embedding ready!


In [14]:
print(bert_eb.bert_embeddings.shape)

(8473, 768)


In [15]:
# controll the consitence of vocabulary
for w in vocab:
    if w not in bert_eb.bert_vocab:
        print(w)

In [16]:
bert_eb.find_similar_words(v, 10, vocab)

{'permanent': 0.9026279354781207,
 'temporarily': 0.892372122495763,
 'temporary': 0.8343887404974022,
 'continuously': 0.8307090696905075,
 'severely': 0.8257456064777264,
 'properly': 0.8235459698434817,
 'automatically': 0.8194902748498845,
 'fixed': 0.8159525263405607,
 'fully': 0.8134978966146471,
 'effectively': 0.8128694718829951}